# OpenSearch の基本概念 - インデックスとドキュメント
> この章は、`index-and-document.ipynb` を元に作成しています。

## ラボの概要
本ラボでは、インデックス、ドキュメントといった OpenSearch の基本的なコンポーネントの操作方法を確認します。

### ラボの構成
本ラボでは、ノートブック環境（EC2 へ Remote Develop 接続した VSCode）および Amazon OpenSearch Serverless を使用します。

<img src="./img/architecture.png" width="50%" style="display: block; margin: auto;">

### 使用するデータセット
本ラボでは、Amazon OpenSearch Service [デベロッパーガイド](https://docs.aws.amazon.com/ja_jp/opensearch-service/latest/developerguide/what-is.html)の[チュートリアル](https://docs.aws.amazon.com/ja_jp/opensearch-service/latest/developerguide/search-example.html)内でも使用している[サンプルムービーデータセット](https://docs.aws.amazon.com/opensearch-service/latest/developerguide/samples/sample-movies.zip)から一部引用して使用しています。


### OpenSearch の基礎知識

#### インデックスとドキュメント

OpenSearch では、検索対象の情報をインデックスに格納します。<br>
インデックスに格納されているデータはドキュメントと呼ばれています。ドキュメントは複数のフィールドを持つ JSON 形式のデータです。非常に噛み砕いた例えではあるのですが、一般的なリレーショナルデータベース (RDB) におけるテーブルとレコード、そしてカラムに近い関係だと考えることができます。

<img src="./img/index_document_field.png" width=1024>

実際のドキュメントは以下のような形式になっています。<br>
アンダースコア(_) が先頭に付与されているフィールドは[メタデータフィールド][response-body-fields]と呼ばれます。ユーザーが登録したドキュメントの内容は _source 配下に格納されています。

```json
{
  "_index": "opensearch_dashboards_sample_data_ecommerce",
  "_id": "p7GpzosBAT8JUQksWpeD",
  "_version": 1,
  "_score": null,
  "_source": {
    "category": [
      "Women's Clothing",
      "Women's Accessories"
    ],
    "currency": "EUR",
    "customer_first_name": "Wilhemina St.",
    "customer_full_name": "Wilhemina St. Riley",
    "customer_gender": "FEMALE",
    "customer_id": 17,
    "customer_last_name": "Riley",
    "customer_phone": "",
    "day_of_week": "Tuesday",
    "day_of_week_i": 1,
    "email": "wilhemina st.@riley-family.zzz",
    "manufacturer": [
      "Microlutions",
      "Pyramidustries"
    ]
}
```

インデックスのドキュメント構造やデータ型は[マッピング][mappings]によって管理されています。<br>
マッピングはリレーショナルデータベースにおけるテーブルスキーマに例えることができるでしょう。OpenSearch で利用可能なすべてのデータ型の一覧は OpenSearch 公式ガイドの [Supported field types][supported-field-types] を参照してください。

[response-body-fields]: https://opensearch.org/docs/latest/api-reference/document-apis/get-documents/#response-body-fields
[mappings]: https://opensearch.org/docs/latest/field-types/mappings-use-cases/
[supported-field-types]: https://opensearch.org/docs/latest/field-types/supported-field-types/index/

#### シャードとノード
インデックスは 1 つ以上のシャードで構成されています。<br>
OpenSearch は 1 つ以上のノードでクラスターを構成する分散アーキテクチャーを採用しています。<br>
OpenSearch はシャードを複数のノードに分散配置することで、データの登録性能や検索性能を、単体ノードの性能を超えてスケールさせることが可能です。<br>
また、シャードはレプリカを設定することも可能です。レプリカを増やすことで、可用性の向上および検索性能のスケールが見込めます。

<img src="./img/shard.png" width=1024>

#### データ型

##### [文字列型](https://opensearch.org/docs/latest/field-types/supported-field-types/string/)
テキスト(文字列)を格納するためのデータ型です。よく使われるものは以下の 2 つです

- text: 文字列を元に、アナライザーによってトークン化や正規化が行われたデータが格納されるフィールドです。全文検索で使用します。
- keyword: 文字列がそのまま格納されてるフィールドです。完全一致検索やファセットの実装に活用できます。ファセットとは、検索結果における特定カテゴリ毎のヒット件数を提供する機能です。ファセットはカテゴリから絞り込み検索を行う場合に有用です。

以下のマッピング例では、検索結果の集計や完全一致検索のためにジャンルは keyword 型に、タイトルは全文検索用に text 型を設定しています。

```json
{
  "mappings": {
    "properties": {
      "title":  { "type": "text" , "analyzer": "standard" },
      "genres":  { "type": "keyword" }
    }
  }
}
```

このほかに、keyword 型の変種で、完全一致検索が遅い代わりにワイルドカード検索の速度を向上させた wildcard 型なども存在します。

##### [数値型](https://opensearch.org/docs/latest/opensearch/supported-field-types/numeric/)

数値データを格納するためのデータ型です。<br>
整数と浮動小数点から選択可能です。 主要なデータ型は以下の通りです。

**整数**

| type          | description             | 
| ------------- | ----------------------- |  
| long          | Signed 64-bit integer   | 
| integer       | Signed 32-bit integer   | 
| short         | Signed 16-bit integer   | 
| byte          | Signed 8-bit integer    | 

**浮動小数点**

| type       | description                                            |
| ---------- | ------------------------------------------------------ | 
| double     | Double precision 64-bit IEEE 754 floating point number |
| float      | Double precision 32-bit IEEE 754 floating point number |
| half_float | Double precision 16-bit IEEE 754 floating point number |

**scaled_float**

float 型で入力されたデータに対して scaling_factor に記述している数値をかけ合わせて丸めた結果を整数として保持します。例えば、**scaling_factor** を `10` に設定した場合、 `2.3` は **23** として内部的に保持されます。<br>
検索時の浮動小数点処理にかかる時間の短縮が見込める他、整数型の方が一般的に圧縮率が高いため、ディスク領域の節約にもなります。

<div class="alert alert-block alert-info"> 
<b>Tips: 数値型の選定指針</b>

基本的には、要件を満たす必要十分なデータ型を選択することを推奨します。<br>
必要十分なデータ型を利用することで、インデクシングと検索のパフォーマンス効率が上がります。<br>
例えば、値の範囲が 0 - 100 に収まる整数を扱う場合は、byte 型を使用します。本ワークショップでも 年を示す year フィールドは short を、rank や上映時間を示す running_time_secs フィールドについては integer を採用しています(一般的に running_time_secs フィールドは short で十分に思えますが、世界には上映時間が数百時間に上る作品もあるとのことで、意図的に integer にしています)。

小数を扱う場合、まずは scaled_float を検討するとよいでしょう。<br>
何らかの理由で scaled_float がフィットしない場合は、必要な桁数に応じて half_float, float, double から選択してください。本ワークショップで作成した index でも rating フィールドが 7.6 といった 0 -10 までの小数点第一位をサポートしたデータであるため、scaling_factor を 10 とした scaled_float タイプを選択しています。

また、数値データだからといって必ずしも数値型を選択することが適切とは限りません。商品の販売価格といった範囲検索や合計、平均といった集計処理に用いる場合は数値データは数値型が望ましいですが、ISBN や製品 ID といった、完全一致検索で用いられるような数値データについては keyword 型の方がフィットします。
</div>

##### [日付型](https://opensearch.org/docs/latest/opensearch/supported-field-types/date/)
エポックミリ秒や ISO など複数のフォーマットをサポートしています。以下のデータ型が利用可能です。

- **date** : ミリ秒単位の日付データをサポート
- **date_nanos**: ナノ秒単位の日付データをサポート。**2262-04-11T23:47:16.854775807** 以降の日付を格納することができない制約があります。

日付型を使用する場合は、格納元のデータと一致する書式を [format](https://opensearch.org/docs/latest/field-types/supported-field-types/date/#formats) オプションで指定します。複数のフォーマットを指定することも可能です。

```json
{
  "mappings": {
    "properties": {
      "date": {
        "type":   "date",
        "format": "yyyyMMdd'T'HHmmss.SSSZ||epoch_millis"
      }
    }
  }
}
```

## 事前作業

### パッケージインストール
実行する前に、タブの右上のカーネルの選択を確認してください。

<div class="alert alert-block alert-warning"> 
opensearch と 後続の章で使用する awswrangler のバージョンの依存関係のため、opensearch-py 側のバージョンを止めています。
</div>

In [ ]:
!uv add "opensearch-py<3" requests-aws4auth python-dotenv

### 環境変数
`.env`ファイルに、以下の環境変数を設定します。

- AOSS_SEARCH_HOST=`AOSS の検索コレクションのエンドポイントのホスト名`
- AOSS_VECTOR_HOST=`AOSS のベクトル検索コレクションのエンドポイントのホスト名`
- AOSS_ROLE_ARN=`AOSS から Bedrock へアクセスするために作成したロールのARN`

設定できたら、以下のコマンドを実行します。

In [ ]:
%reload_ext dotenv
%dotenv -o

### インポート

In [ ]:
import boto3
import json
import logging
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import os

### 共通変数のセット

In [ ]:
default_region = boto3.Session().region_name
logging.getLogger().setLevel(logging.ERROR)

### OpenSearch Serverless への接続確認
OpenSearch Server のセキュリティ設定により、API リクエストが許可されているかを確認します。

In [ ]:
aoss_host = os.getenv("AOSS_SEARCH_HOST")

credentials = boto3.Session().get_credentials()
service_code = "aoss"
auth = AWSV4SignerAuth(credentials=credentials, region=default_region, service=service_code)
opensearch_client = OpenSearch(
    hosts=[{"host": aoss_host, "port": 443}],
    http_compress=True, 
    http_auth=auth,
    use_ssl=True,
    verify_certs=True,
    connection_class = RequestsHttpConnection
)
opensearch_client.cat.indices()

## インデックスの作成
以下のフィールドを持つインデックスを作成します。

| フィールド | 説明 | フィールドタイプ | 備考 |
|---|---|---|---|
| id | 映画の ID | [keyword][keyword] | ドキュメント ID にも使用します。|
| directors | 監督名 | [text][text] | |
| release_date | リリース日 | [date][date] ||
| rating | 0.0 から 10.0 までの評価値 | [scaled_float][numeric] ||
| genres | ジャンル名 | [keyword][keyword] ||
| image_url | 画像の URL | [keyword][keyword] ||
| plot | あらすじ | [text][text] ||
| title | タイトル | [text][text] ||
| rank | ランキング | [integer][numeric] ||
| running_time_secs | 上映時間 | [integer][numeric] ||
| actors | 俳優名 | [text][text] ||
| year | リリースされた年 | [short][numeric] ||
| type | 操作タイプ | [keyword][keyword] ||


インデックスの作成は [Create index API](https://opensearch.org/docs/latest/api-reference/index-apis/create-index/) で行います。<br>
Python クライアントを使用する場合、[Indices Client](https://opensearch-project.github.io/opensearch-py/api-ref/clients/indices_client.html) を使用することも可能です。本ワークショップではこれ以降、基本的にインデックスの操作やドキュメントの操作は Indices Client を使用して行います

[text]: https://opensearch.org/docs/latest/field-types/supported-field-types/text/
[date]: https://opensearch.org/docs/latest/field-types/supported-field-types/date/
[scaled_float]: https://opensearch.org/docs/latest/field-types/supported-field-types/numeric/#scaled-float-field-type
[keyword]: https://opensearch.org/docs/latest/field-types/supported-field-types/keyword/
[wildcard]: https://opensearch.org/docs/latest/field-types/supported-field-types/wildcard/
[numeric]: https://opensearch.org/docs/latest/field-types/supported-field-types/numeric/


In [ ]:
index_name = "movies"

payload = {
  "settings": {
    "index": {
      "number_of_shards": 1,
      "number_of_replicas": 0
    }
  },
  "mappings": {
    "properties": {
      "id":  { "type": "keyword" },
      "directors":  { "type": "text" },
      "release_date": {"type": "date"},
      "rating": {"type": "scaled_float", "scaling_factor": 10},
      "genres":  { "type": "keyword" },
      "image_url": { "type": "keyword" },
      "plot":  { "type": "text" },
      "title":  { "type": "text" },
      "rank": { "type": "integer" },
      "running_time_secs": { "type": "integer" },
      "actors":  { "type": "text" },
      "year": {"type": "short"},
      "type": { "type": "keyword" } 
    }
  }
}

try:
    # 既に同名のインデックスが存在する場合、いったん削除を行う
    print("# delete index")
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

# インデックスの作成を行う
print("# create index")
response = opensearch_client.indices.create(index=index_name, body=payload)
print(json.dumps(response, indent=2))

## 単一ドキュメントの操作
OpenSearch のドキュメント操作は、インデックスの作成と同様に API を通じて行います。

### ドキュメント作成
ドキュメントの作成には、[Index document API][index-document] を使用します。<br>
Indices client を用いてドキュメント登録を行う場合は、インデックス名を index パラメーターに、ドキュメント本文を body に与えます。ドキュメント ID の付与は任意ですが、本ワークショップでは明示的に指定しています。refresh オプションは、ドキュメント書き込み後に [リフレッシュ(Refresh)][refresh] を明示的に実行するかを指定するものです。リフレッシュはバッファ内のドキュメントデータをファイル(厳密にはファイルシステムキャッシュ)に書き出すことで、ドキュメントを検索可能とするための操作です。True にセットした場合、ドキュメントが速やかに検索可能となります。

> AOSS では、refresh はサポートされていません。

<div class="alert alert-block alert-info"> 
<b>Tips: ドキュメント ID を明示的に指定するべきか</b>

ドキュメント作成時に ID を指定しない場合、OpenSearch クラスターによって自動的に ID が生成されます。ID はインデックス内で一意です。検索機能をアプリケーションに組み込むようなケースでは、ドキュメントの更新が発生しうるため、ドキュメント ID には意味のある文字列(ユーザー ID 等)を指定するのが一般的な設計です。一方、ログデータのような更新が発生しないデータについては、自動採番を用いる方がパフォーマンスの面で有利です。明示的に ID を指定してドキュメントの作成を行う場合は ID の衝突チェックが行われるのに対して、自動採番はチェックをスキップするためです。
</div>

[index-document]: https://opensearch.org/docs/latest/api-reference/document-apis/index-document/
[refresh]: https://opensearch.org/docs/latest/api-reference/index-apis/refresh/

<div class="alert alert-block alert-warning"> 
AOSS では、検索コレクションの場合は、ドキュメント ID を使用できますが、ベクトル検索コレクションの場合は、ドキュメント ID を指定した登録はできません。自動採番されたドキュメント ID を使用することはできます。
</div>

In [ ]:
index_name = "movies"
payload = {
  "id": "tt2229499",
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.4,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2012,
  "type": "add"
}

response = opensearch_client.index(
    index = index_name,
    body = payload,
    id = "tt2229499",
    # AOSS では指定できません。
    # refresh = True
)

print(json.dumps(response, indent=2))

#### ドキュメント作成結果の確認
ドキュメント作成の成否は、上記レスポンスより確認することが可能です。<br>
**result** フィールドの値が `created` となっていれば作成に成功したと判断できます。<br>
作成直後のドキュメントのバージョンはデフォルトだと 1 となります。**_version** フィールドよりドキュメントのバージョンが確認できます。

### ドキュメント参照
作成したドキュメントは、インデックス名と ID を指定して [Get document API][get-documents] を発行することで参照可能です

[get-documents]: https://opensearch.org/docs/latest/api-reference/document-apis/get-documents

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)

print(json.dumps(response, indent=2))

### ドキュメント更新
ドキュメントの更新方法は上書きと部分更新の二通りあります。各更新方法について解説します。

#### 完全上書きによる更新
先ほど入力したデータですが、実は year = 2013 が正しい値でした。<br>
year フィールドの値を修正していきましょう。<br>
完全上書きでドキュメントを更新する場合は、作成時と同様の [Index document API][index-document] を用います。完全上書きの場合、更新の有無によらず全てのフィールドをペイロードに含める必要があります。

[index-document]: https://opensearch.org/docs/latest/api-reference/document-apis/index-document/

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "id": document_id,
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.0,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2013,
  "type": "add"
}

response = opensearch_client.index(
    index = index_name,
    body = payload,
    id = document_id,
    # AOSS では指定できません。
    # refresh = True
)

print(json.dumps(response, indent=2))

更新されたドキュメントを確認すると、_version および _seq_no が 1 ずつ増加していることが確認できます。<br>
これらのメタデータはドキュメント固有のもので、バージョンチェックに基づく楽観的なロックを行いたい場合に用いられます。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)

print(json.dumps(response, indent=2))

#### 部分更新・部分追加
先程の更新で rating の値が 7.4 から 7.0 に意図せず変わってしまいました。<br>こちらの値だけを修正していきましょう。<br>
ドキュメントの部分更新には [Update document API][update-document] を使用します。

[update-document]: https://opensearch.org/docs/latest/api-reference/document-apis/update-document/

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "doc": {
    "rating": 7.4,
  }
}

response = opensearch_client.update(
    index = index_name,
    body = payload,
    id = document_id,
    # AOSS では指定できません。
    # refresh = True
)

print(json.dumps(response, indent=2))

Get document API により、部分更新の結果が反映されていることが確認できます。

> ここでも、refresh できないので、即時には `7.4` になっていない場合があります。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)

print(json.dumps(response, indent=2))

#### ドキュメントの上書き更新を禁止する
Index document API 実行時に op_type パラメーターで create を指定することで、ドキュメントが存在する場合にエラーを返すことができます。上書を禁止したいユースケースでは、デフォルトの index ではなく create を明示的に指定してください。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "id": document_id,
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.0,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2013,
  "type": "add"
}


try:
    response = opensearch_client.index(
        index = index_name,
        body = payload,
        id = document_id,
        op_type="create",
        # AOSS では指定できません。
        # refresh = True
    )
except Exception as e:
    print(e)

#### バージョンによるドキュメント上書きの禁止

OpenSearch ではドキュメントに **_version** というメタデータが付与されています。<br>
このメタデータを書き込み時に使用することで、古いバージョンによる上書きのみを防ぐことができます。<br>
_version にデータ発生時のタイムスタンプなどを用いることで、過去データの上書きによるデータの巻き戻りを防ぐことができます。


In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)

current_document_version = response["_version"]

print(json.dumps(response, indent=2))

ドキュメント更新時にバージョンによるチェックを行う場合は、version_type オプションおよび version オプションを追加します。リクエストで指定するバージョンが現行のドキュメントバージョンより大きい場合のみ書き込みを許可する場合は、**version_type** に `external` を指定します。

以下は **version_type** に `external` を指定し、**version** に現行バージョンと同じバージョン値を入力した結果、書き込みエラーとなった例です。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

# Update 
payload = {
  "id": document_id,
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.0,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2013,
  "type": "add"
}

try:
    response = opensearch_client.index(
        index = index_name,
        body = payload,
        id = document_id,
        version = current_document_version,
        version_type = "external",
        op_type="index",
        # AOSS では指定できません。
        # refresh = True
    )
    print(json.dumps(response, indent=2))
    current_document_version = response["_version"]
except Exception as e:
    print(e)

以下は現在のバージョンより 1 大きいバージョンを指定することで、書き込みに成功する例です。

In [ ]:
new_document_version = current_document_version + 1
index_name = "movies"
document_id = "tt2229499"

payload = {
  "id": document_id,
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.0,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2013,
  "type": "add"
}


try:
    response = opensearch_client.index(
        index = index_name,
        body = payload,
        id = document_id,
        version = new_document_version,
        version_type = "external",
        op_type="index",
        # AOSS では指定できません。
        # refresh = True
    )
    current_document_version = response["_version"]
    print(json.dumps(response, indent=2))
    
except Exception as e:
    print(e)


**version_type** に `external_gte` を指定することで、同一バージョンによる上書を許可することも可能です。

In [ ]:
new_document_version = current_document_version
index_name = "movies"
document_id = "tt2229499"

payload = {
  "id": document_id,
  "directors": ["Joseph Gordon-Levitt"],
  "release_date": "2013-01-18T00:00:00Z",
  "rating": 7.0,
  "genres": ["Comedy","Drama"],
  "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
  "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
  "title": "Don Jon",
  "rank": 1,
  "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
  "year": 2013,
  "type": "add"
}


try:
    response = opensearch_client.index(
        index = index_name,
        body = payload,
        id = document_id,
        version = new_document_version,
        version_type = "external_gte",
        op_type="index",
        # AOSS では指定できません。
        # refresh = True
    )
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

#### 楽観的ロックによる書き込み制御
ドキュメントの更新を行う際、script により既存値をベースとした更新を行うことができます。<br>
以下は現状の rating 値に + 0.1 を行った結果を新しい rating として更新を行う処理のサンプルです。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)
print(json.dumps(response, indent=2))

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "script" : {
    "source": "ctx._source.rating += params.delta",
    "lang": "painless",
    "params" : {
      "delta" : 0.1
    }
  }
}

response = opensearch_client.update(
    index = index_name,
    body = payload,
    id = document_id,
    # AOSS では指定できません。
    # refresh = True
)

print(json.dumps(response, indent=2))

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)
print(json.dumps(response, indent=2))

現在値ベースの更新処理を実行する場合、他のリクエストにより予期せずデータが更新された場合は処理をキャンセルしたいという要件があります。

こうした要件に対応するために、ドキュメントの[メタデータ][metadata] である **_seq_no** および **primary_term** を利用します。<br>
_seq_no はインデックス固有のカウンタです。ドキュメントの追加、削除、更新を行うごとにカウントが 1 ずつ増加する性質をもっています。<br>
ドキュメントの作成または更新を行うと、その時点におけるインデックスの _seq_no がドキュメントにも記録されます。何回目(シーケンスの何番目)の操作でドキュメントが作成または更新されたかが記録されるということです。

primary_term は、OpenSearch のシャード(プライマリシャード)がアサインされた回数を指します。このカウンタはノード障害等でレプリカシャードがプライマリシャードに昇格した際に増加します。<br>
この 2 つの値を使用して、ドキュメントが予期せず更新された際にエラーとみなすことができます。

[metadata]: https://opensearch.org/docs/latest/api-reference/document-apis/get-documents/#response-body-fields

以下のサンプルでは、クエリ実行時に現在の _seq_no を正しく if_seq_no パラメーターにセットしています。この場合、クエリ実行に成功します。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.get(
    index = index_name,
    id = document_id
)
seq_no = response["_seq_no"]
primary_term = response["_primary_term"]

payload = {
  "script" : {
    "source": "ctx._source.rating += params.delta",
    "lang": "painless",
    "params" : {
      "delta" : 0.1
    }
  }
}

try:
    response = opensearch_client.update(
        index = index_name,
        body = payload,
        id = document_id,
        # AOSS では指定できません。
        # refresh = True,
        if_seq_no = seq_no,
        if_primary_term = primary_term
    )
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)


以下のサンプルでは、クエリ実行時に前回の _seq_no をそのまま if_seq_no パラメーターにセットしています。この場合、_seq_no が一致していないためクエリ実行に失敗します。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "script" : {
    "source": "ctx._source.rating += params.delta",
    "lang": "painless",
    "params" : {
      "delta" : 0.1
    }
  }
}

try:
    response = opensearch_client.update(
        index = index_name,
        body = payload,
        id = document_id,
        # AOSS では指定できません。
        # refresh = True,
        if_seq_no = seq_no,
        if_primary_term = primary_term
    )
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

### ドキュメント削除
[Delete document API][delete-document] を使用することでドキュメント削除が可能です。

[delete-document]: https://opensearch.org/docs/latest/api-reference/document-apis/delete-document/

In [ ]:
index_name = "movies"
document_id = "tt2229499"

response = opensearch_client.delete(
    index = index_name,
    id = document_id,
    # AOSS では指定できません。
    # refresh = True
)

print(json.dumps(response, indent=2))

存在しないドキュメントに対して GET リクエストを実行すると、 404 エラーが返却されることが確認できます。これにより削除判定が行えます。

> ここも Refresh していないので、即時でやると取得できてしまいます。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

try:
    response = opensearch_client.get(
        index = index_name,
        id = document_id
    )
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)

### ドキュメントの Upsert

OpenSearch では UPSERT 処理も実行することが可能です。UPSERT とは UPDATE と INSERT を組み合わせた言葉です。指定のデータが存在すれば更新を、存在しなければ新規作成を行うことを指します。

デフォルトでは、Update document API は存在しないドキュメントに対する更新処理を実行できません。404 エラーが返却されます。

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "doc": {
    "id": document_id,
    "directors": ["Joseph Gordon-Levitt"],
    "release_date": "2013-01-18T00:00:00Z",
    "rating": 7.0,
    "genres": ["Comedy","Drama"],
    "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
    "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
    "title": "Don Jon",
    "rank": 1,
    "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
    "year": 2013,
    "type": "add"
  }
}

try:
    response = opensearch_client.update(
        index = index_name,
        body = payload,
        id = document_id,
        # AOSS では指定できません。
        # refresh = True
    )
except Exception as e:
    print(e)

**doc_as_upsert** オプションを付与した Update document API を実行することで、 [Upsert][upsert] 相当の処理を行えます。

[upsert]: https://opensearch.org/docs/latest/api-reference/document-apis/update-document/#upsert

In [ ]:
index_name = "movies"
document_id = "tt2229499"

payload = {
  "doc": {
    "id": document_id,
    "directors": ["Joseph Gordon-Levitt"],
    "release_date": "2013-01-18T00:00:00Z",
    "rating": 7.0,
    "genres": ["Comedy","Drama"],
    "image_url": "https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg",
    "plot": "A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.",
    "title": "Don Jon",
    "rank": 1,
    "actor": ["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],
    "year": 2013,
    "type": "add"
  },
  "doc_as_upsert": True
}

try:
    response = opensearch_client.update(
        index = index_name,
        body = payload,
        id = document_id,
        # AOSS では指定できません。
        # refresh = True
    )
except Exception as e:
    print(e)
    
print(json.dumps(response, indent=2))

## 複数ドキュメントの一括処理
OpenSearch では [Bulk API][bulk] を使用することで、単一の API リクエストに複数のドキュメント操作を含めることが可能です。都度 API リクエストを発行する場合と比較して、処理効率が向上します。

Bulk API は 1 行以上の JSON 文字列で 1 つのオペレーションを実行します。 1 行目にオペレーション内容、書き込み対象の Index 、ドキュメント ID を記載し、2行目にドキュメントの実データを記載します。2 行目の要否および書式はオペレーションの内容によって変化します。1 行目で _id を指定しない場合、対象のドキュメントには OpenSearch が作成したランダムな ID が割り当てられます。

```json
{ "<operation>" : { "_index" : "<index-name>", "_id" : "<id>" } }
{ "field1" : "value1", "field2": "value2", ... }
```

Bulk API で 利用可能なオペレーションは以下の通りです。

- create: ドキュメントの作成を行います。<br>同一 ID のドキュメントが存在する場合、エラーを返します。Index document API の op_type = create に相当します。
- index: ドキュメントの作成を行います。<br>同一 ID のドキュメントが存在する場合、上書きを行います。Index document API の デフォルト挙動 (op_type = index) に相当します。
- update: ドキュメントの部分更新を行います。<br>doc_as_upsert オプションを true にセットした場合、upsert 処理が実行されます。Update document API に相当します。
- delete: ドキュメントの削除を行います。<br>Delete document API に相当します。

Bulk API は個々のオペレーション操作の成否に関わらず、一連の処理が完了した時点でレスポンスコード 200 と合わせて結果を返却します。 全体を通してのエラー有無は errors から判断可能です。errors が true である場合、個々のオペレーションの成否を確認していきます。 個々のオペレーションの成否については、各オペレーションの status や error から判断します。

[bulk]: https://opensearch.org/docs/latest/api-reference/document-apis/bulk/

In [ ]:
payload = """
{ "delete" : { "_index" : "movies", "_id" : "tt2229499" } }
{ "delete" : { "_index" : "movies", "_id" : "tt2229499" } }
{ "create": { "_index": "movies", "_id": "tt2229499" } }
{"directors":["Joseph Gordon-Levitt"],"release_date":"2013-01-18T00:00:00Z","rating":7.4,"genres":["Comedy","Drama"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg","plot":"A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.","title":"Don Jon","rank":1,"running_time_secs":5400,"actors":["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],"year":2012,"id":"tt2229499","type":"add"}
{ "index": { "_index": "movies", "_id": "tt1979320" } }
{"directors":["Ron Howard"],"release_date":"2013-09-02T00:00:00Z","rating":8.3,"genres":["Action","Biography","Drama","Sport"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTQyMDE0MTY0OV5BMl5BanBnXkFtZTcwMjI2OTI0OQ@@._V1_SX400_.jpg","plot":"A re-creation of the merciless 1970s rivalry between Formula One rivals James Hunt and Niki Lauda.","title":"Rush","rank":2,"running_time_secs":7380,"actors":["Daniel Brühl","Chris Hemsworth","Olivia Wilde"],"year":2013,"id":"tt1979320","type":"add"}
{ "create": { "_index": "movies", "_id": "tt2229499" } }
{"directors":["Joseph Gordon-Levitt"],"release_date":"2013-01-18T00:00:00Z","rating":7.4,"genres":["Comedy","Drama"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg","plot":"A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.","title":"Don Jon","rank":1,"running_time_secs":5400,"actors":["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],"year":2013,"id":"tt2229499","type":"add"}
{ "index": { "_index": "movies", "_id": "tt2229499" } }
{"directors":["Joseph Gordon-Levitt"],"release_date":"2013-01-18T00:00:00Z","rating":7.0,"genres":["Comedy","Drama"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTQxNTc3NDM2MF5BMl5BanBnXkFtZTcwNzQ5NTQ3OQ@@._V1_SX400_.jpg","plot":"A New Jersey guy dedicated to his family, friends, and church, develops unrealistic expectations from watching porn and works to find happiness and intimacy with his potential true love.","title":"Don Jon","rank":1,"running_time_secs":5400,"actors":["Joseph Gordon-Levitt","Scarlett Johansson","Julianne Moore"],"year":2013,"id":"tt2229499","type":"add"}
{ "update" : {"_id" : "tt2229499", "_index" : "movies"} }
{ "doc" : {"rating" : 7.4}, "doc_as_upsert": true }
{ "index": { "_index": "movies", "_id": "tt1392214" } }
{"directors":["Denis Villeneuve"],"release_date":"2013-08-30T00:00:00Z","rating":8.2,"genres":["Crime","Drama","Thriller"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTg0NTIzMjQ1NV5BMl5BanBnXkFtZTcwNDc3MzM5OQ@@._V1_SX400_.jpg","plot":"When Keller Dover's daughter and her friend go missing, he takes matters into his own hands as the police pursue multiple leads and the pressure mounts. But just how far will this desperate father go to protect his family?","title":"Prisoners","rank":3,"running_time_secs":9180,"actors":["Hugh Jackman","Jake Gyllenhaal","Viola Davis"],"year":2013,"id":"tt1392214","type":"add"}
{ "index": { "_index": "movies", "_id": "tt1981115" } }
{"directors":["Alan Taylor"],"release_date":"2013-10-30T00:00:00Z","genres":["Action","Adventure","Fantasy"],"image_url":"https://m.media-amazon.com/images/M/MV5BMTQyNzAwOTUxOF5BMl5BanBnXkFtZTcwMTE0OTc5OQ@@._V1_SX400_.jpg","plot":"Faced with an enemy that even Odin and Asgard cannot withstand, Thor must embark on his most perilous and personal journey yet, one that will reunite him with Jane Foster and force him to sacrifice everything to save us all.","title":"Thor: The Dark World","rank":5,"year":2013,"actors":["Chris Hemsworth","Natalie Portman","Tom Hiddleston"],"id":"tt1981115","type":"add"}
"""

response = opensearch_client.bulk(body=payload, refresh=False)

print(json.dumps(response, indent=2))

### 複数ドキュメントの取得

OpenSearch では [Multi get][multi-get] API を使用することで、単一のリクエストで 1 つ以上のインデックスから複数のドキュメントを取得することが可能です。都度 API リクエストを発行する場合と比較して、処理効率が向上します。

[multi-get]: https://opensearch.org/docs/latest/api-reference/document-apis/multi-get/

In [ ]:
index_name = "movies"

payload = {
  "docs": [
    {
      "_index": index_name,
      "_id": "tt2229499"
    },
    {
      "_index": index_name,
      "_id": "tt1979320"
    }
  ]
}

response = opensearch_client.mget(
    body=payload
)

print(json.dumps(response, indent=2))

## まとめ
本ラボでは、OpenSearch の基本概念について解説しました。本ラボで学習した内容を元に、次のステップとして以下のラボを実行してみましょう。

- [OpenSearch の基本的な検索機能](./2-basic-search.ipynb)

## 後片付け

### インデックス削除
本ワークショップで使用したインデックスを削除します。インデックスの削除は Delete index API で行います。インデックスを削除するとインデックス内のドキュメントも削除されます。

In [ ]:
index_name = "movies"

try:
    response = opensearch_client.indices.delete(index=index_name)
    print(json.dumps(response, indent=2))
except Exception as e:
    print(e)